Welcome!

This is a daily report, on the performance of specific exchange rates. It is to be ran at market close, to determine potential courses of action.

In [19]:
###Historical price report (outdated, from yesterday's closing to 2 days!)

import pyodbc
from datetime import datetime
import pandas as pd
import yfinance as yf
import requests
import pandas_ta as ta
import pyodbc

global dictdate
dictdate = {
    "AXS" : "2020-11-04",
    "ENS" : "2021-11-09",
    "NEXO": "2018-05-01",
    "DOT" : "2020-08-20",
    "XRP" : "2017-11-09",
    "ETH" : "2017-11-09",
    "ARB11841" : "2023-03-23",
    "TIA22861" : "2023-10-31",
    "ADA" : "2017-11-09",
    "HBAR" : "2019-09-17",
    "SOL" : "2020-04-10",
    "NEAR" : "2020-10-14",
    "RUNE" : "2019-07-23",
    "FTM" : "2018-10-30",
    "LINK" : "2017-11-09",
    "BCH" : "2017-11-09",
    "AVAX" : "2020-07-13",
    "ORDI" : "2023-05-08",
    "OP" : "2022-03-14",
    "DOGE" : "2017-11-09",
    "BONK" : "2022-12-30",
    "MATIC" : "2019-04-28",
    "AI" : "2021-09-22",
    "LTC" : "2014-09-17",
    "BNB" : "2017-11-09",
    "ETC" : "2017-11-09",
    "JUP" : "2017-11-09",
    "XMR" : "2017-11-09",
    "FIL" : "2017-12-13",
    "DYDX" : "2021-11-08",
    "ATOM" : "2019-03-14",
    "EOS" : "2017-11-09",
    "TRB" : "2019-11-19",
    "FTM" : "2018-10-30",
    "JTO" : "2023-12-07",
    "GALA" : "2020-09-18",
    "BLUR" : "2023-02-14"
}

def dataImport(token):

    global creation
    global current
    global activetime

    if token in dictdate.keys():
        datestr = dictdate[token]
    elif token == "ARB":
        datestr = "2023-03-23"
        token = "ARB11841"
    elif token == "TIA":
        datestr = "2023-10-31"
        token = "TIA22861"
    else:
        datestr = input("Not in our database! Enter a date (YYYY-MM-DD): ")
        print ("Don't forget to add the date to the dictionary!")

    creation = datetime.strptime(datestr, "%Y-%m-%d")
    current = datetime.today()
    activetime = (current - creation).days

    # Connection parameters - replace with your values
    server = 'PERSONALDEVICEG' 
    database = 'models'
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';Trusted_Connection=yes;')

    cursor = cnxn.cursor()

    # Set up the table and the auto procedure

    try:
        # It's more common to use separate execute calls for each SQL statement, but here's a try for batch execution
        cursor.execute(f"""
            IF OBJECT_ID('{token}daily', 'P') IS NOT NULL
                DROP PROCEDURE {token}daily;
        """)

        cursor.execute(f"""            
            IF OBJECT_ID('{token}HistoricalPrices', 'U') IS NOT NULL
                DROP TABLE {token}HistoricalPrices;
        """)

        cursor.execute(f"""
            CREATE TABLE {token}HistoricalPrices (
                [Date] DATE,
                [Open] DECIMAL(10, 6),
                [High] DECIMAL(10, 6),
                [Low] DECIMAL(10, 6),
                [Close] DECIMAL(10, 6),
                [Adj Close] DECIMAL(10, 6),
                [Volume] BIGINT
            );
        """)

        cursor.execute(f"""
            CREATE PROCEDURE {token}daily 
                @dates DATE,
                @opens DECIMAL(10, 6),
                @highs DECIMAL(10, 6),
                @lows DECIMAL(10, 6),
                @closes DECIMAL(10, 6),
                @adjcloses DECIMAL(10, 6),
                @volumes BIGINT
            AS BEGIN
                INSERT INTO {token}HistoricalPrices (
                    [Date],
                    [Open],
                    [High],
                    [Low],
                    [Close],
                    [Adj Close],
                    [Volume]
                )
                VALUES (
                    @dates,
                    @opens,
                    @highs,
                    @lows,
                    @closes,
                    @adjcloses,
                    @volumes
                );
            END;
        """)

        cnxn.commit()

    except Exception as e:
        print(f'Preliminary SQL execution failed: {e}')
        cursor.close()
        cnxn.close()
        return


    # Read data from Yahoo! Finance

    today_date = datetime.today().strftime('%Y-%m-%d')

    df = yf.download(f"{token}-USD", start=datestr, end=today_date)

    df.reset_index(inplace=True)

    # Loop through the DataFrame and insert each row
    for index, row in df.iterrows():
        cursor.execute(f"EXEC {token}daily @dates = ?, @opens = ?, @highs = ?, @lows = ?, @closes = ?, @adjcloses = ?, @volumes = ?", 
                       row['Date'], row['Open'], row['High'], row['Low'], row['Close'], row['Adj Close'], row['Volume'])

    # Commit the transaction
    try :
        cnxn.commit()
    except:
        print ('Data import failed!')

    # Close the connection
    cursor.close()
    cnxn.close()



# Function to calculate the daily percentage change in price and volume
def calculate_price_volume_change(df):
    # Calculate daily percentage change in price
    df['Price_Change'] = df['Close'].pct_change() * 100
    
    # Calculate daily percentage change in volume
    df['Volume_Change'] = df['Volume'].pct_change() * 100
    
    return df

# Function to identify potential pump-and-dump activities based on predefined thresholds
def identify_potential_pump_and_dump(df, price_change_threshold, volume_change_threshold):
    # Identify potential pump-and-dump activities
    df['Potential_Pump_and_Dump'] = (df['Price_Change'] > price_change_threshold) & (df['Volume_Change'] > volume_change_threshold)
    
    return df

# Function to conduct market context analysis incorporating pump-and-dump analysis
def conduct_market_context_analysis(df):
    # Initialize market context column
    df['Market_Context'] = 'Neutral'
    
    # Update market context based on pump-and-dump analysis
    df.loc[df['Potential_Pump_and_Dump'], 'Market_Context'] = 'Potential Pump-and-Dump'
    
    return df

def emotional(token, index):

    pd.set_option('display.max_columns', None)

    cnxn = connectionEst()

    query = f"SELECT * FROM {token}HistoricalPrices"
    df = pd.read_sql(query, cnxn)

    if token == "ARB":
        token = "ARB11841"

    if token == "TIA":
        token = "TIA22861"


    # Define thresholds for identifying potential pump-and-dump activities
    price_change_threshold = 10  # Percentage change in price threshold
    volume_change_threshold = 20  # Percentage change in volume threshold

    query = f"SELECT * FROM {token}HistoricalPrices"
    df = pd.read_sql(query, cnxn)

    # Calculate price and volume changes
    df = calculate_price_volume_change(df)

    # Identify potential pump-and-dump activities
    df = identify_potential_pump_and_dump(df, price_change_threshold, volume_change_threshold)

    # Conduct market context analysis
    df = conduct_market_context_analysis(df)

    df['Index'] = index

    # Display the DataFrame with updated market context
    return df.tail(1).sort_values(by='Date', ascending = False)

def connectionEst():

    global cnxn

    # Connection parameters - replace with your values
    server = 'PERSONALDEVICEG' 
    database = 'models'
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';Trusted_Connection=yes;')
    return cnxn


def calculate_technical_indicators(df, index, period=20, num_std=2):
    # Assuming df has 'Close', 'High', and 'Low' columns

    # Convert 'Date' to datetime and set as index

    # Calculate Moving Averages
    df['SMA_20'] = ta.sma(df['Close'], length=period)
    df['EMA_20'] = ta.ema(df['Close'], length=period)
   
    # Bollinger Bands
    df['SMA'] = df['Close'].rolling(window=period).mean()  # Simple Moving Average
    df['STD'] = df['Close'].rolling(window=period).std()  # Standard Deviation
    df['Upper_Band'] = df['SMA'] + (df['STD'] * num_std)
    df['Lower_Band'] = df['SMA'] - (df['STD'] * num_std)

    # RSI
    df['RSI_14'] = ta.rsi(df['Close'], length=14)

    # MACD
    macd = ta.macd(df['Close'], fast=12, slow=26, signal=9)
    try:
        df['MACD'] = macd['MACD_12_26_9']
        df['MACD_Signal'] = macd['MACDs_12_26_9']
        df['MACD_Histogram'] = macd['MACDh_12_26_9']
    except:
        df['MACD'] = "Not available"
        df['MACD_Signal'] = "Not available"
        df['MACD_Histogram'] = "Not available"
    
    # Identifying bullish or bearish trends
    try:
        df['Trend'] = 'Neutral'
        df.loc[df['Close'] > df['SMA_20'], 'Trend'] = 'Bullish'
        df.loc[df['Close'] < df['SMA_20'], 'Trend'] = 'Bearish'
    except:
        df['Trend'] = "Not available"


    # Volume confirmation
    df['Volume_Confirmation'] = False
    df.loc[(df['Volume'].shift(1) < df['Volume']) & 
           ((df['Trend'] == 'Bullish') & (df['Close'] > df['Close'].shift(1)) | 
            (df['Trend'] == 'Bearish') & (df['Close'] < df['Close'].shift(1))), 'Volume_Confirmation'] = True

    # Bollinger Bands for volatility and price position
    df['Bollinger_Signal'] = 'Neutral'
    df.loc[df['Close'] > df['Upper_Band'], 'Bollinger_Signal'] = 'Overbought'
    df.loc[df['Close'] < df['Lower_Band'], 'Bollinger_Signal'] = 'Oversold'

    # RSI for momentum
    df['RSI_Signal'] = 'Neutral'
    df.loc[df['RSI_14'] > 70, 'RSI_Signal'] = 'Overbought'
    df.loc[df['RSI_14'] < 30, 'RSI_Signal'] = 'Oversold'

    # MACD for trend changes
    df['MACD_Signal_Char'] = 'Neutral'
    df.loc[df['MACD'] > df['MACD_Signal'], 'MACD_Signal_Char'] = 'Bullish'
    df.loc[df['MACD'] < df['MACD_Signal'], 'MACD_Signal_Char'] = 'Bearish'

    # Price action relative to moving averages for potential reversals
    try:
        df['Price_MA_Relation'] = 'Neutral'
        df.loc[(df['Close'] > df['SMA_20']) & (df['Close'].shift(1) < df['SMA_20'].shift(1)), 'Price_MA_Relation'] = 'Potential Reversal Up'
        df.loc[(df['Close'] < df['SMA_20']) & (df['Close'].shift(1) > df['SMA_20'].shift(1)), 'Price_MA_Relation'] = 'Potential Reversal Down'
    except:
         df['Price_MA_Relation'] = 'Not available'

    try:
        df['Trend_Volume'] = 'Neutral'
        df.loc[(df['Close'] > df['SMA_20']) & (df['Close'] > df['EMA_20']) & (df['Volume'].diff() > 0), 'Trend_Volume'] = 'Bullish with Volume Increase'
        df.loc[(df['Close'] < df['SMA_20']) & (df['Close'] < df['EMA_20']) & (df['Volume'].diff() > 0), 'Trend_Volume'] = 'Bearish with Volume Increase'
    except:
        df['Trend_Volume'] = 'Not available'
    
    # Bollinger Bands and Volume for Market Extremes
    df['Bollinger_Volume'] = 'Neutral'
    df.loc[(df['Close'] > df['Upper_Band']) & (df['Volume'].diff() > 0), 'Bollinger_Volume'] = 'Overbought with Volume Increase'
    df.loc[(df['Close'] < df['Lower_Band']) & (df['Volume'].diff() > 0), 'Bollinger_Volume'] = 'Oversold with Volume Increase'
    
    # RSI and Volume for Momentum Confirmation
    df['RSI_Volume'] = 'Neutral'
    df.loc[(df['RSI_14'] > 70) & (df['Volume'].diff() > 0), 'RSI_Volume'] = 'Overbought with High Volume'
    df.loc[(df['RSI_14'] < 30) & (df['Volume'].diff() > 0), 'RSI_Volume'] = 'Oversold with High Volume'
    
    # MACD, Volume, and Market Reversals
    df['MACD_Signal_Volume'] = 'Neutral'
    df.loc[(df['MACD'] > df['MACD_Signal']) & (df['Volume'].diff() > 0), 'MACD_Signal_Volume'] = 'Bullish Crossover with Volume Increase'
    df.loc[(df['MACD'] < df['MACD_Signal']) & (df['Volume'].diff() > 0), 'MACD_Signal_Volume'] = 'Bearish Crossover with Volume Increase'
    
    # Contextual Analysis Based on Historical Highs/Lows
    try:
        df['Market_Context'] = 'Neutral'
        df.loc[(df['Close'] > df['SMA_20']) & (df['Close'] > df['EMA_20']) & (df['Close'] < df['Low'].min() * 1.10), 'Market_Context'] = 'Bullish Near Historical Lows'
        df.loc[(df['Close'] < df['SMA_20']) & (df['Close'] < df['EMA_20']) & (df['Close'] > df['High'].max() * 0.90), 'Market_Context'] = 'Bearish Near Historical Highs'
    except:
        df['Market_Context'] = 'Not available'

    df ['Index'] = index

    return df.tail(1).sort_values(by='Date', ascending = False)


def technical(token, index):

    pd.set_option('display.max_columns', None)

    cnxn = connectionEst()

    query = f"SELECT * FROM {token}HistoricalPrices"
    df = pd.read_sql(query, cnxn)

    if df.empty:
        return

    return calculate_technical_indicators(df, index)



emotionalFrame = None
technicalFrame = None
appendix = None
new = None
index = None

for k in dictdate.keys():

    dataImport(k)

for k in dictdate.keys():

    if emotionalFrame is None:
        new = emotional (k, k)
        emotionalFrame = new
    else:
        appendix = emotional(k, k)
        emotionalFrame = pd.concat((emotionalFrame, appendix), axis = 0)

for k in dictdate.keys():

    if technicalFrame is None:
        new = technical(k, k)
        technicalFrame = new
    else:
        appendix = technical(k, k)
        technicalFrame = pd.concat((technicalFrame, appendix), axis = 0)

complete = pd.merge(emotionalFrame, technicalFrame, on = 'Index')

complete

    

[*********************100%%**********************]  1 of 1 completed

[*********************100%%**********************]  1 of 1 completed

[*********************100%%**********************]  1 of 1 completed

[*********************100%%**********************]  1 of 1 completed

[*********************100%%**********************]  1 of 1 completed

...

<ipython-input-19-5aa5bc7624e2>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)
<ipython-input-19-5aa5bc7624e2>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)
<ipython-input-19-5aa5bc7624e2>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)
<ipython-input-19-5aa5bc7624e2>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects ar

<ipython-input-19-5aa5bc7624e2>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)
<ipython-input-19-5aa5bc7624e2>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)
<ipython-input-19-5aa5bc7624e2>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)
<ipython-input-19-5aa5bc7624e2>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects ar

<ipython-input-19-5aa5bc7624e2>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)
<ipython-input-19-5aa5bc7624e2>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)
<ipython-input-19-5aa5bc7624e2>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)
<ipython-input-19-5aa5bc7624e2>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects ar

<ipython-input-19-5aa5bc7624e2>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)
<ipython-input-19-5aa5bc7624e2>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)
<ipython-input-19-5aa5bc7624e2>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)
<ipython-input-19-5aa5bc7624e2>:356: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects ar

,Date_x,Open_x,High_x,Low_x,Close_x,Adj Close_x,Volume_x,Price_Change,Volume_Change,Potential_Pump_and_Dump,Market_Context_x,Index,Date_y,Open_y,High_y,Low_y,Close_y,Adj Close_y,Volume_y,SMA_20,EMA_20,SMA,STD,Upper_Band,Lower_Band,RSI_14,MACD,MACD_Signal,MACD_Histogram,Trend,Volume_Confirmation,Bollinger_Signal,RSI_Signal,MACD_Signal_Char,Price_MA_Relation,Trend_Volume,Bollinger_Volume,RSI_Volume,MACD_Signal_Volume,Market_Context_y
0,2024-06-17,6.709504,6.753820,6.075200,6.236229,6.236229,58972589,-7.043714,146.445097,False,Neutral,AXS,2024-06-17,6.709504,6.753820,6.075200,6.236229,6.236229,58972589,7.456303,7.230368,7.456303,0.646051,8.748404,6.164202,34.204125,-0.295463,-1.424767e-01,-1.529860e-01,Bearish,True,Neutral,Neutral,Bearish,Neutral,Bearish with Volume Increase,Neutral,Neutral,Bearish Crossover with Volume Increase,Neutral
1,2024-06-17,24.446024,24.446024,22.183582,22.713413,22.713413,129559995,-7.097457,39.965525,False,Neutral,ENS,2024-06-17,24.446024,24.446024,22.183582,22.713413,22.713413,129559995,23.378281,22.280358,23.378281,2.599615,28.577512,18.179050,52.427745,0.403865,6.035829e-01,-1.997179e-01,Bearish,True,Neutral,Neutral,Bearish,Potential Reversal Down,Neutral,Neutral,Neutral,Bearish Crossover with Volume Increase,Neutral
2,2024-06-17,1.349742,1.353197,1.305200,1.315984,1.315984,4827270,-2.503310,19.301816,False,Neutral,NEXO,2024-06-17,1.349742,1.353197,1.305200,1.315984,1.315984,4827270,1.414920,1.388545,1.414920,0.051123,1.517167,1.312673,37.748115,-0.009929,8.539554e-03,-1.846857e-02,Bearish,True,Neutral,Neutral,Bearish,Neutral,Bearish with Volume Increase,Neutral,Neutral,Bearish Crossover with Volume Increase,Neutral
3,2024-06-17,6.387890,6.408695,6.005823,6.073915,6.073915,185556256,-4.917809,41.887849,False,Neutral,DOT,2024-06-17,6.387890,6.408695,6.005823,6.073915,6.073915,185556256,6.709866,6.626261,6.709866,0.385889,7.481645,5.938087,36.579123,-0.249097,-1.886948e-01,-6.040234e-02,Bearish,True,Neutral,Neutral,Bearish,Neutral,Bearish with Volume Increase,Neutral,Neutral,Bearish Crossover with Volume Increase,Neutral
4,2024-06-17,0.488814,0.519142,0.484668,0.505160,0.505160,2028523218,3.340207,193.124178,False,Neutral,XRP,2024-06-17,0.488814,0.519142,0.484668,0.505160,0.505160,2028523218,0.503999,0.501213,0.503999,0.017097,0.538193,0.469806,49.980712,-0.009085,-8.775188e-03,-3.102788e-04,Bullish,True,Neutral,Neutral,Bearish,Potential Reversal Up,Bullish with Volume Increase,Neutral,Neutral,Bearish Crossover with Volume Increase,Neutral
5,2024-06-17,3622.383545,3634.285400,3468.148926,3511.378906,3511.378906,17838856988,-3.015679,80.584694,False,Neutral,ETH,2024-06-17,3622.383545,3634.285400,3468.148926,3511.378906,3511.378906,17838856988,3677.806677,3603.510790,3677.806677,125.576687,3928.960051,3426.653303,45.801969,4.838774,4.439311e+01,-3.955434e+01,Bearish,True,Neutral,Neutral,Bearish,Neutral,Bearish with Volume Increase,Neutral,Neutral,Bearish Crossover with Volume Increase,Neutral
6,2024-06-17,0.923674,0.928739,0.846415,0.858712,0.858712,258986764,-7.033101,99.729144,False,Neutral,ARB11841,2024-06-17,0.923674,0.928739,0.846415,0.858712,0.858712,258986764,1.021388,0.994124,1.021388,0.094277,1.209942,0.832833,29.870862,-0.057134,-4.155144e-02,-1.558210e-02,Bearish,True,Neutral,Oversold,Bearish,Neutral,Bearish with Volume Increase,Neutral,Oversold with High Volume,Bearish Crossover with Volume Increase,Neutral
7,2024-06-17,8.192727,8.313493,7.146741,7.258075,7.258075,109990211,-11.385708,129.481622,False,Neutral,TIA22861,2024-06-17,8.192727,8.313493,7.146741,7.258075,7.258075,109990211,9.502028,8.919686,9.502028,1.382738,12.267504,6.736552,33.512654,-0.640778,-3.851474e-01,-2.556303e-01,Bearish,True,Neutral,Neutral,Bearish,Neutral,Bearish with Volume Increase,Neutral,Neutral,Bearish Crossover with Volume Increase,Neutral
8,2024-06-17,0.416388,0.419987,0.393655,0.401758,0.401758,385694374,-3.511464,123.069966,False,Neutral,ADA,2024-06-17,0.416388,0.419987,0.393655,0.401758,0.401758,385694374,0.4

In [ ]:
#Current price import


import ccxt
import schedule
import time
import pyodbc
from datetime import datetime
import pandas as pd
import yfinance as yf
import requests
import pandas_ta as ta
import pyodbc

global dictdate
dictdate = {
    "AXS" : "2020-11-04",
    "ENS" : "2021-11-09",
    "NEXO": "2018-05-01",
    "DOT" : "2020-08-20",
    "XRP" : "2017-11-09",
    "ETH" : "2017-11-09",
    "ARB11841" : "2023-03-23",
    "TIA22861" : "2023-10-31",
    "ADA" : "2017-11-09",
    "HBAR" : "2019-09-17",
    "SOL" : "2020-04-10",
    "NEAR" : "2020-10-14",
    "RUNE" : "2019-07-23",
    "FTM" : "2018-10-30",
    "LINK" : "2017-11-09",
    "BCH" : "2017-11-09",
    "AVAX" : "2020-07-13",
    "ORDI" : "2023-05-08",
    "OP" : "2022-03-14",
    "DOGE" : "2017-11-09",
    "BONK" : "2022-12-30",
    "MATIC" : "2019-04-28",
    "AI" : "2021-09-22",
    "LTC" : "2014-09-17",
    "BNB" : "2017-11-09",
    "ETC" : "2017-11-09",
    "JUP" : "2017-11-09",
    "XMR" : "2017-11-09",
    "FIL" : "2017-12-13",
    "DYDX" : "2021-11-08",
    "ATOM" : "2019-03-14",
    "EOS" : "2017-11-09",
    "TRB" : "2019-11-19",
    "FTM" : "2018-10-30",
    "JTO" : "2023-12-07",
    "GALA" : "2020-09-18",
    "BLUR" : "2023-02-14"
}



def currentPrice():

    global iterno
    iterno = 1
    server = 'PERSONALDEVICEG' 
    database = 'models'
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';Trusted_Connection=yes;')
    cursor = cnxn.cursor()

    for key in dictdate.keys():
        try:
            # Ensure the table exists
            cursor.execute(f"""
                IF OBJECT_ID('{key}CurrentPrices', 'U') IS NULL
                CREATE TABLE {key}CurrentPrices (
                    [Date] DATE,
                    [Time] TIME,
                    [Price] DECIMAL(10, 6),
                    [Volume] BIGINT
                );
            """)
            cnxn.commit()

            # Ensure the procedure exists
            cursor.execute(f"""
                IF OBJECT_ID('{key}current', 'P') IS NULL
                BEGIN
                    EXEC('CREATE PROCEDURE {key}current 
                        @dates DATE,
                        @times TIME,
                        @prices DECIMAL(10, 6),
                        @volumes BIGINT
                        AS
                        BEGIN
                            INSERT INTO {key}CurrentPrices (
                                [Date],
                                [Time],
                                [Price],
                                [Volume]
                            )
                            VALUES (
                                @dates,
                                @times,
                                @prices,
                                @volumes
                            );
                        END;');
                END;
            """)
            cnxn.commit()

            # Fetch data from CCXT
            exchange = ccxt.binance()
            exchange.load_markets(reload=True)
            ticker = exchange.fetch_ticker(f'{key}/USDT')
            price = ticker['last']  # Fetching last price
            volume = ticker['quoteVolume']

            # Execute the stored procedure with actual data
            cursor.execute(f"EXEC {key}current @dates=?, @times=?, @prices=?, @volumes=?", 
                           (datetime.now().strftime('%Y-%m-%d'), datetime.now().strftime('%H:%M:%S'), price, volume))
            cnxn.commit()

            print(f"Completed {iterno}/{len(dictdate.keys())} currencies")
            iterno += 1

        except Exception as e:
            print(f'Error processing {key}: {e}')
            continue  # Continue to next key in case of an error

    cursor.close()
    cnxn.close()
    print("Round complete!")

# Schedule and run the task
schedule.clear()
schedule.every(1).minutes.do(currentPrice)

while True:
    schedule.run_pending()
    time.sleep(1)  # Prevent high CPU usage



Completed 9/36 currencies


In [ ]:
#Monitoring algorithm for currency of trends


#Connect to the database and retrieve a dataframe with the historical data
#Then connect again and retrieve a dataframe with the actual data after closing
#Choose a validation outcome based on observed trends
#Check if it has happened:

##SMA

    # Calculate Moving Averages
    df['SMA_20'] = ta.sma(df['Close'], length=period)


##EMA


    df['EMA_20'] = ta.ema(df['Close'], length=period)


##Bollinger bands
   
    # Bollinger Bands
    df['SMA'] = df['Close'].rolling(window=period).mean()  # Simple Moving Average
    df['STD'] = df['Close'].rolling(window=period).std()  # Standard Deviation
    df['Upper_Band'] = df['SMA'] + (df['STD'] * num_std)
    df['Lower_Band'] = df['SMA'] - (df['STD'] * num_std)


#Oversell-overbuy signals

    # RSI
    df['RSI_14'] = ta.rsi(df['Close'], length=14)

    # MACD
    macd = ta.macd(df['Close'], fast=12, slow=26, signal=9)
    try:
        df['MACD'] = macd['MACD_12_26_9']
        df['MACD_Signal'] = macd['MACDs_12_26_9']
        df['MACD_Histogram'] = macd['MACDh_12_26_9']
    except:
        df['MACD'] = "Not available"
        df['MACD_Signal'] = "Not available"
        df['MACD_Histogram'] = "Not available"
    
    # Identifying bullish or bearish trends
    try:
        df['Trend'] = 'Neutral'
        df.loc[df['Close'] > df['SMA_20'], 'Trend'] = 'Bullish'
        df.loc[df['Close'] < df['SMA_20'], 'Trend'] = 'Bearish'
    except:
        df['Trend'] = "Not available"

    df['Volume_Confirmation'] = False
    df.loc[(df['Volume'].shift(1) < df['Volume']) & 
           ((df['Trend'] == 'Bullish') & (df['Close'] > df['Close'].shift(1)) | 
            (df['Trend'] == 'Bearish') & (df['Close'] < df['Close'].shift(1))), 'Volume_Confirmation'] = True

    df['Bollinger_Signal'] = 'Neutral'
    df.loc[df['Close'] > df['Upper_Band'], 'Bollinger_Signal'] = 'Overbought'
    df.loc[df['Close'] < df['Lower_Band'], 'Bollinger_Signal'] = 'Oversold'

    df['RSI_Signal'] = 'Neutral'
    df.loc[df['RSI_14'] > 70, 'RSI_Signal'] = 'Overbought'
    df.loc[df['RSI_14'] < 30, 'RSI_Signal'] = 'Oversold'


##MACD trend

    # MACD for trend changes
    df['MACD_Signal_Char'] = 'Neutral'
    df.loc[df['MACD'] > df['MACD_Signal'], 'MACD_Signal_Char'] = 'Bullish'
    df.loc[df['MACD'] < df['MACD_Signal'], 'MACD_Signal_Char'] = 'Bearish'



##Potential reversal up-down

    try:
        df['Price_MA_Relation'] = 'Neutral'
        df.loc[(df['Close'] > df['SMA_20']) & (df['Close'].shift(1) < df['SMA_20'].shift(1)), 'Price_MA_Relation'] = 'Potential Reversal Up'
        df.loc[(df['Close'] < df['SMA_20']) & (df['Close'].shift(1) > df['SMA_20'].shift(1)), 'Price_MA_Relation'] = 'Potential Reversal Down'
    except:
         df['Price_MA_Relation'] = 'Not available'

    try:
        df['Trend_Volume'] = 'Neutral'
        df.loc[(df['Close'] > df['SMA_20']) & (df['Close'] > df['EMA_20']) & (df['Volume'].diff() > 0), 'Trend_Volume'] = 'Bullish with Volume Increase'
        df.loc[(df['Close'] < df['SMA_20']) & (df['Close'] < df['EMA_20']) & (df['Volume'].diff() > 0), 'Trend_Volume'] = 'Bearish with Volume Increase'
    except:
        df['Trend_Volume'] = 'Not available'


##Overbought/oversold and volume increase
    
    # Bollinger Bands and Volume for Market Extremes
    df['Bollinger_Volume'] = 'Neutral'
    df.loc[(df['Close'] > df['Upper_Band']) & (df['Volume'].diff() > 0), 'Bollinger_Volume'] = 'Overbought with Volume Increase'
    df.loc[(df['Close'] < df['Lower_Band']) & (df['Volume'].diff() > 0), 'Bollinger_Volume'] = 'Oversold with Volume Increase'


##Momemntum maintained/reversed
    
    # RSI and Volume for Momentum Confirmation
    df['RSI_Volume'] = 'Neutral'
    df.loc[(df['RSI_14'] > 70) & (df['Volume'].diff() > 0), 'RSI_Volume'] = 'Overbought with High Volume'
    df.loc[(df['RSI_14'] < 30) & (df['Volume'].diff() > 0), 'RSI_Volume'] = 'Oversold with High Volume'


##Reversal pending/happened
    
    # MACD, Volume, and Market Reversals
    df['MACD_Signal_Volume'] = 'Neutral'
    df.loc[(df['MACD'] > df['MACD_Signal']) & (df['Volume'].diff() > 0), 'MACD_Signal_Volume'] = 'Bullish Crossover with Volume Increase'
    df.loc[(df['MACD'] < df['MACD_Signal']) & (df['Volume'].diff() > 0), 'MACD_Signal_Volume'] = 'Bearish Crossover with Volume Increase'


##Historical high/low against price movement

    
    # Contextual Analysis Based on Historical Highs/Lows
    try:
        df['Market_Context'] = 'Neutral'
        df.loc[(df['Close'] > df['SMA_20']) & (df['Close'] > df['EMA_20']) & (df['Close'] < df['Low'].min() * 1.10), 'Market_Context'] = 'Bullish Near Historical Lows'
        df.loc[(df['Close'] < df['SMA_20']) & (df['Close'] < df['EMA_20']) & (df['Close'] > df['High'].max() * 0.90), 'Market_Context'] = 'Bearish Near Historical Highs'
    except:
        df['Market_Context'] = 'Not available'